[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/lenguajenatural-ai/autotransformers/blob/master/notebooks/chatbot_instructions/somosnlp24_entrenamiento_instrucciones.ipynb)

# Hackathon SomosNLP 2024: Entrenamiento de LLMs

En este tutorial veremos cómo entrenar LLMs para instrucciones / chat con las herramientas de HuggingFace. En la siguiente parte del notebook veremos cómo hacer esto mismo con [autotransformers](https://github.com/lenguajenatural-ai/autotransformers), añadiendo el entrenamiento con NEFTune.

Lo primero de todo instalamos la librería `autotransformers` que ya nos va a traer directamente el resto de dependencias que necesitamos.

In [ ]:
!pip install autotransformers

## Importación de Librerías

Este bloque de código se encarga de importar todas las librerías necesarias para el funcionamiento del script. Se importan herramientas para la manipulación de modelos de aprendizaje automático como `torch` y `transformers`, así como librerías específicas para la preparación y configuración de modelos (`peft`), carga y procesamiento de conjuntos de datos (`datasets`), y una librería especial (`trl`) para el entrenamiento de modelos de lenguaje mediante técnicas de fine-tuning.

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig, TrainingArguments,  AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer
import torch
from peft.tuners.lora import LoraLayer

## Creando la plantilla de chat

En esta sección, se crea una plantilla para formatear los mensajes de chat durante el entrenamiento. La plantilla utiliza sintaxis específica para identificar y organizar los roles de los participantes en la conversación (usuario, sistema, asistente, entrada), permitiendo que el modelo comprenda y genere respuestas adecuadas dentro del contexto establecido.

In [ ]:
# create chat template
CHAT_TEMPLATE = """{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{'<user> ' + message['content'].strip() + ' </user>' }}
    {% elif message['role'] == 'system' %}
        {{'<system>\\n' + message['content'].strip() + '\\n</system>\\n\\n' }}
    {% elif message['role'] == 'assistant' %}
        {{ message['content'].strip() + ' </assistant>' + eos_token }}
    {% elif message['role'] == 'input' %}
        {{'<input> ' + message['content'] + ' </input>' }}
    {% endif %}
{% endfor %}"""

## Carga del dataset y preprocesado

Se carga un dataset específico llamado `somosnlp/somos-clean-alpaca-es` usando la librería `datasets`. Posteriormente, se define y aplica una función de preprocesado (`process_alpaca`) que estructura cada muestra del dataset en un formato adecuado para entrenar chatbots, etiquetando cada mensaje con su respectivo rol en la conversación. Finalmente, el dataset procesado se divide en conjuntos de entrenamiento y prueba.

In [ ]:
alpaca = load_dataset("somosnlp/somos-clean-alpaca-es")

def process_alpaca(sample: dict) -> dict:
    """
    Processes a single sample from the alpaca dataset to structure it for chatbot training.

    This function transforms the dataset sample into a format suitable for training,
    where each message is categorized by its role in the conversation (system, input, user, assistant).
    It initializes the conversation with a system message, then conditionally adds an input message,
    follows with the user's instruction, and finally, the assistant's output based on the provided inputs.

    Parameters
    ----------
    sample : dict
        A dictionary representing a single sample from the dataset. It must contain
        keys corresponding to input and output components of the conversation.

    Returns
    -------
    dict
        A modified dictionary with a 'messages' key that contains a list of ordered messages,
        each annotated with its role in the conversation.
    """
    chat = [
        {"role": "system", "content": "Eres un asistente que resuelve las instrucciones del usuario. Si se proporciona contexto adicional, utiliza esa información para completar la instrucción."}
    ]
    inp_ = sample["inputs"]["2-input"] 
    if inp_ is not None and inp_ != "":
        chat.append(
            {"role": "input", "content": inp_}
        )
    chat.extend(
        [
            {"role": "user", "content": sample["inputs"]["1-instruction"]},
            {"role": "assistant", "content": sample["inputs"]["3-output"]}
        ]
    )
    sample["messages"] = chat
    return sample

alpaca = alpaca.map(
    process_alpaca,
    batched=False,
    num_proc=4,
    remove_columns=[col for col in alpaca["train"].column_names if col != "messages"])

alpaca = alpaca["train"].train_test_split(0.2, seed=203984)

## Definición de los argumentos de entrenamiento

Se configuran los argumentos de entrenamiento utilizando la clase `TrainingArguments` de la librería `transformers`. Estos argumentos incluyen configuraciones importantes como el tamaño del batch, la tasa de aprendizaje, el tipo de optimizador, y varios otros parámetros que influencian directamente en el rendimiento y la eficiencia del entrenamiento del modelo.


In [ ]:
training_args = TrainingArguments(
    output_dir="./gemma_2b_alpaca",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_ratio=0.03,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=50,
    lr_scheduler_type="constant",
    weight_decay=0.001,
    eval_steps=200,
    save_steps=50,
    num_train_epochs=1,
    logging_first_step=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    max_grad_norm=0.3,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    group_by_length=False,
    save_total_limit=5,
)

## Carga del tokenizador

Se carga un tokenizador preentrenado correspondiente al modelo `google/gemma-2b` usando la librería `transformers`. Además, se configura el tokenizador con la plantilla de chat creada anteriormente y se ajustan parámetros específicos como el token de relleno y la longitud máxima de secuencia.


In [ ]:
model_name = "google/gemma-2b"
max_seq_length = 4096
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.model_max_length = max_seq_length
tokenizer.chat_template=CHAT_TEMPLATE

## Función de formateo del chat

Esta función toma las muestras del dataset y las procesa aplicando la plantilla de chat configurada previamente. El objetivo es tokenizar las entradas para que el modelo pueda entender y generar respuestas durante el entrenamiento y la evaluación.

In [ ]:
def format_chat(
    samples: dict,
) -> dict:
    """
    Tokenize inputs for chatbot or instruction tuning.

    Parameters
    ----------
    samples: Dict
        Dataset samples to process.

    Returns
    -------
    samples: Dict
        Processed samples with tokenized data.
    """
    texts = []
    for i in range(len(samples["messages"])):
        full_text = tokenizer.apply_chat_template(
            samples["messages"][i], tokenize=False
        )
        texts.append(full_text)
    return texts

## Carga del modelo

Se configura y carga el modelo de lenguaje causal para entrenamiento con cuantización y ajustes específicos para mejorar el rendimiento y reducir el consumo de memoria. Se utiliza una configuración específica para LoRA (Low-Rank Adaptation) y QLoRA (Quantized LoRA), ajustando parámetros como el rango y la tasa de dropout, y se prepara el modelo para el entrenamiento con estos ajustes.

In [ ]:
lora_config = LoraConfig(
    r=64, # NOTE: Al usar rslora podemos subir el rango con mejoras en el rendimiento.
    lora_alpha=32,
    target_modules="all-linear", # NOTE: En QLoRA entrenamos todas las capas lineales del modelo.
    lora_dropout=0.10,  # 0.1 for <13B models, 0.05 otherwise.
    bias="none",
    task_type="CAUSAL_LM",
    use_rslora=True # NOTE: flag para usar QLoRA.
)

In [ ]:

qlora_config = BitsAndBytesConfig(
    load_in_4bit=True, # NOTE: Lo cargamos en 4bits.
    bnb_4bit_use_double_quant=True, # NOTE: Usamos la doble cuantización de QLoRA para ahorrar aún más espacio.
    bnb_4bit_quant_type="nf4", # NOTE: Usamos NormalFloat 4bits ya que según el paper de QLoRA funciona mejor.
    bnb_4bit_compute_dtype=torch.bfloat16, # NOTE: Utilizamos para los cálculos bfloat16; cambiar a float16 en arquitecturas no Ampere.
)


In [ ]:

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=qlora_config, token=True)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model = get_peft_model(model, lora_config)
model.config.use_cache = False


In [ ]:
for name, module in model.named_modules():
    if isinstance(module, LoraLayer):
        module = module.to(torch.bfloat16)
    if "norm" in name:
        module = module.to(torch.float32)
    if "lm_head" in name or "embed_tokens" in name:
        if hasattr(module, "weight"):
            module = module.to(torch.bfloat16)

# Definición del Trainer y Entrenamiento

Se inicializa el `Trainer` (en este caso un `SFTTrainer` específico para entrenamiento de modelos de lenguaje) con el modelo, los argumentos de entrenamiento, y el dataset formateado. Finalmente, se ejecuta el entrenamiento del modelo utilizando el método `.train()`.

In [ ]:
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=alpaca["train"],
    eval_dataset=alpaca["test"],
    formatting_func=format_chat,
    max_seq_length=max_seq_length
)

In [ ]:
trainer.train()

# Entrenamiento de LLMs con AutoTransformers.

Ahora veremos cómo llevar a cabo esto mismo con `autotransformers`, que simplifica el proceso a la vez que ofrece mayor flexibilidad en cómo se procesan los datos y se lleva a cabo el entrenamiento. Esta parte es una adaptación a español de [este notebook](https://github.com/lenguajenatural-ai/autotransformers/blob/master/notebooks/chatbot_instructions/train_instructional_chatbot.ipynb), que tiene las explicaciones más completas desarrolladas originalmente en inglés.

In [1]:
from autotransformers import AutoTrainer, DatasetConfig, ModelConfig
from autotransformers.llm_templates import instructions_to_chat, NEFTuneTrainer, QLoraWrapperModelInit, modify_tokenizer, qlora_config, SavePeftModelCallback
from functools import partial
from datasets import load_dataset
from peft import LoraConfig

[nltk_data] Downloading package punkt to /home/alejandro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/alejandro/miniconda3/envs/test_autotransformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/alejandro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Creando la Plantilla de Chat

Para formatear correctamente las conversaciones para el entrenamiento, definimos una plantilla de chat usando la sintaxis de plantillas Jinja2. Esta plantilla itera a través de cada mensaje en una conversación, categorizándolos y formateándolos basados en su rol:

- **Mensajes de Usuario**: Envueltos con etiquetas `<user>` para indicar claramente mensajes del usuario. Estos son las instrucciones o consultas dirigidas al chatbot.

- **Mensajes del Sistema**: Encerrados dentro de etiquetas `<system>`, seguidos por saltos de línea para la legibilidad. Estos mensajes podrían incluir instrucciones generadas por el sistema o contexto que guía las respuestas del chatbot.

- **Respuestas del Asistente**: Colocadas entre la conversación, después de las etiquetas `</user>` y marcadas con etiquetas `</assistant>` al final, junto con el token de fin de oración (EOS). Estas son las respuestas del chatbot o acciones tomadas en respuesta al mensaje del usuario, en cada intervención o turno en la conversación.

- **Datos de Entrada**: Marcados con etiquetas `<input>` para distinguir cualquier entrada adicional o información contextual proporcionada al chatbot.

Este formato estructurado es crucial para que el modelo entienda los diferentes componentes de una conversación, permitiéndole generar respuestas apropiadas basadas en el rol de cada mensaje.

Típicamente, una conversación empezará con el mensaje del sistema, luego tendrá una entrada conteniendo contexto adicional para el asistente, y luego turnos de usuario-asistente, que pueden ser uno o más.


In [2]:
CHAT_TEMPLATE = """{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{'<user> ' + message['content'].strip() + ' </user>' }}
    {% elif message['role'] == 'system' %}
        {{'<system>\\n' + message['content'].strip() + '\\n</system>\\n\\n' }}
    {% elif message['role'] == 'assistant' %}
        {{ message['content'].strip() + ' </assistant>' + eos_token }}
    {% elif message['role'] == 'input' %}
        {{'<input> ' + message['content'] + ' </input>' }}
    {% endif %}
{% endfor %}"""

## Preparación del Dataset

La fase de preparación del dataset es crucial para estructurar los datos de manera que sea propicia para el entrenamiento de un chatbot. Primero cargamos el dataset desde el hub y luego utilizamos `instructions_to_chat`, para transformar cada muestra del dataset `somos-clean-alpaca` en un formato que refleje un flujo de conversación real involucrando un mensaje del sistema, la entrada del usuario y la respuesta del asistente.

### La Función `instructions_to_chat`

`instructions_to_chat` toma un diccionario que representa una sola muestra del dataset y lo reestructura categorizando y ordenando mensajes basados en su rol en una conversación:

- Comienza agregando un **mensaje del sistema** que establece el contexto para el chatbot como un asistente diseñado para seguir las instrucciones del usuario.
- Si está presente, los **datos de entrada** se agregan a continuación para proporcionar contexto o información adicional necesaria para cumplir con la solicitud del usuario.
- La **instrucción del usuario** se añade luego, seguida de la **respuesta del asistente**, que es la respuesta a la solicitud del usuario.

Esta reestructuración resulta en una lista `messages` dentro del diccionario de muestra, conteniendo todos los elementos de la conversación en su orden lógico.

### Aplicando la Transformación

Para aplicar esta transformación a través de todo el dataset:

- Utilizamos el método `.map` con `instructions_to_chat` como la función de mapeo, estableciendo `batched=False` para procesar las muestras individualmente y `num_proc=4` para paralelizar la operación, mejorando la eficiencia.
- Se eliminan las columnas que no forman parte de la estructura de `messages` para simplificar el dataset.

Finalmente, el dataset se divide en conjuntos de entrenamiento y prueba con un 20% para el tamaño de prueba, asegurando que podamos evaluar el rendimiento de nuestro chatbot en datos no vistos. Esta división se logra usando el método `train_test_split`, proporcionando una base sólida para entrenar y validar el modelo del chatbot.

In [3]:
alpaca = load_dataset("somosnlp/somos-clean-alpaca-es")

In [4]:
alpaca = alpaca.map(
    partial(
        instructions_to_chat,
        input_field="1-instruction",
        context_field="2-input",
        output_field="3-output",
        nested_field="inputs",
        system_message="Eres un asistente que resuelve las instrucciones que le presenta el usuario. En caso de tener un contexto adicional, utilízalo para resolver la instrucción."
    ),
    batched=False,
    num_proc=4,
    remove_columns=[col for col in alpaca["train"].column_names if col != "messages"])

In [5]:
alpaca = alpaca["train"].train_test_split(0.2, seed=203984)

## Configurando el Dataset para AutoTransformers

Para asegurar que nuestro modelo de chatbot instructivo se entrene de manera eficiente y efectiva, configuramos meticulosamente nuestro dataset usando la configuración de dataset (`DatasetConfig`) de la biblioteca `autotransformers`. Este paso es esencial para adaptar el proceso de entrenamiento a nuestras necesidades específicas, incluyendo la configuración de hiperparámetros, detalles del dataset y estrategias de entrenamiento.

### Configuración de los Argumentos de Entrenamiento

Se define un conjunto de argumentos de entrenamiento fijos (`fixed_train_args`) para controlar varios aspectos del proceso de entrenamiento:

- **Tamaños de lote** tanto para el entrenamiento como para la evaluación se establecen en 1, indicando que las muestras se procesan individualmente. Esto puede ser particularmente útil para modelos grandes o cuando la memoria GPU es limitada.
- **Acumulación de gradientes** se utiliza con 16 pasos, permitiéndonos simular efectivamente un tamaño de lote más grande y estabilizar el entrenamiento sin exceder los límites de memoria.
- Un **ratio de calentamiento** de 0.03 aumenta gradualmente la tasa de aprendizaje al comienzo del entrenamiento para prevenir que el modelo converja demasiado rápido a una solución subóptima.
- **Tasa de aprendizaje**, **decaimiento de peso**, y otros ajustes de optimización son cuidadosamente elegidos para equilibrar la velocidad de aprendizaje del modelo y la calidad.
- **Estrategias de evaluación y guardado** se configuran para verificar periódicamente el rendimiento del modelo y guardar puntos de control, permitiendo el monitoreo y la continuación del entrenamiento desde el último estado guardado.

### Creando la Configuración del Dataset

El diccionario `alpaca_config` abarca toda la información necesaria para la preparación e integración del dataset:

- **Detalles del dataset** como el nombre, tipo de tarea y columnas específicas a usar para texto y etiquetas aseguran que el modelo se entrene en el formato correcto de datos.
- **Parámetros de entrenamiento** se incluyen a través del diccionario `fixed_training_args`.
- **Clases de callback**, como `SavePeftModelCallback`, automatizan pasos importantes como el guardado del modelo durante el entrenamiento.
- **Optimizaciones de proceso** como establecer una semilla para reproducibilidad, especificar la dirección de optimización y la métrica, y habilitar divisiones parciales para la creación del conjunto de validación.


In [6]:
fixed_train_args = {
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 16,
    "warmup_ratio": 0.03,
    "learning_rate": 2e-4,
    "bf16": True,
    "logging_steps": 50,
    "lr_scheduler_type": "constant",
    "weight_decay": 0.001,
    "eval_steps": 200,
    "save_steps": 50,
    "num_train_epochs": 1,
    "logging_first_step": True,
    "evaluation_strategy": "steps",
    "save_strategy": "steps",
    "max_grad_norm": 0.3,
    "optim": "paged_adamw_32bit",
    "gradient_checkpointing": True,
    "group_by_length": False,
    "save_total_limit": 50,
    "adam_beta2": 0.999
}

In [7]:
alpaca_config = {
    "seed": 9834,
    "callbacks": [SavePeftModelCallback],
    "fixed_training_args": fixed_train_args,
    "dataset_name": "alpaca",
    "alias": "alpaca",
    "retrain_at_end": False,
    "task": "chatbot",
    "text_field": "messages",
    "label_col": "messages",
    "num_proc": 4, # 
    "loaded_dataset": alpaca, # Aquí metemos el dataset pre-cargado.
    "partial_split": True, # NOTE: Para crear una partición de validación.
}

In [8]:
alpaca_config = DatasetConfig(**alpaca_config)

## Configuración del Modelo

En la sección "Configuración del Modelo", delineamos cómo configurar las configuraciones del modelo usando `autotransformers`, enfocándonos en integrar LoRA (Adaptación de Bajo Rango) para la adaptación del modelo y aplicar la cuantización para la eficiencia. Estos pasos son cruciales para personalizar el modelo para nuestra tarea y entorno específicos, asegurando un rendimiento óptimo y la utilización de recursos.

### Configuración de LoRA

El objeto `LoraConfig` se instancia con parámetros diseñados para mejorar la adaptabilidad del modelo mientras se mantiene la eficiencia:

- **r (rango)** y **lora_alpha** se establecen para ajustar la capacidad y el multiplicador de la tasa de aprendizaje para las capas LoRA, equilibrando entre la flexibilidad del modelo y el riesgo de sobreajuste.
- **target_modules** especifica qué partes del modelo aplicar LoRA. En este caso, se apuntan los módulos "all-linear" para la adaptación, ofreciendo una mejora amplia sobre las capacidades del modelo.
- **lora_dropout** se ajusta según el tamaño del modelo, asegurando que la regularización esté escalada apropiadamente.
- La configuración de **bias** se establece en "none", indicando que no se usan términos de bias adicionales en las capas de adaptación LoRA.
- El **task_type** se especifica como "CAUSAL_LM" para indicar la tarea de modelado del lenguaje causal, alineándose con la naturaleza del chatbot instructivo.
- El parámetro **use_rslora** se utiliza para activar rank-stabilized lora que nos permite entrenar con rangos más altos.

### Configuración del Modelo GEMMA

La `ModelConfig` para el modelo GEMMA incluye varios parámetros clave y personalizaciones:

- **Nombre del Modelo**: Especifica el modelo preentrenado a ser adaptado, "google/gemma-2b-it" en este caso.
- **Nombre de Guardado y Directorio**: Define la convención de nomenclatura y ubicación para guardar el modelo afinado.
- **Parámetros Personalizados**: Incluye configuraciones específicas del modelo, como habilitar la confianza en código remoto y configurar el mapeo de dispositivos para el entrenamiento.
- **Envoltorio de Inicialización del Modelo**: `QLoraWrapperModelInit` se usa para integrar el marco de cuantización QLoRA con el modelo configurado LoRA, optimizando tanto la adaptabilidad como la eficiencia.
- **Configuraciones de Cantidadización y PEFT**: Se aplican a través de los parámetros `quantization_config` y `peft_config`, asegurando que el modelo se beneficie tanto de las adaptaciones LoRA como de la cuantización eficiente después del entrenamiento.
- **Modificación del Tokenizador**: Se usa una función parcial para personalizar el tokenizador, ajustando la longitud de secuencia, añadiendo tokens especiales e incorporando la plantilla de chat diseñada para nuestro contexto conversacional.


In [9]:
lora_config = LoraConfig(
    r=64, # NOTE: Al usar rslora podemos subir el rango con mejoras en el rendimiento.
    lora_alpha=32,
    target_modules="all-linear", # NOTE: En QLoRA entrenamos todas las capas lineales del modelo.
    lora_dropout=0.10,  # NOTE: 0.1 for <13B models, 0.05 otherwise.
    bias="none",
    task_type="CAUSAL_LM",
    use_rslora=True # NOTE: flag para usar QLoRA.
)

In [10]:
gemma_config = ModelConfig(
    name="google/gemma-2b-it",
    save_name="gemma_2b",
    save_dir="./gemma_2b_alpaca",
    model_init_wrap_cls=QLoraWrapperModelInit,
    quantization_config=qlora_config,
    peft_config=lora_config,
    neftune_noise_alpha=10, # NOTE: Este es el parámetro que podemos tocar de NEFTune.
    custom_trainer_cls=NEFTuneTrainer, # NOTE: Un Trainer ajustado para usar NEFTune.
    func_modify_tokenizer=partial(
        modify_tokenizer,
        new_model_seq_length=4096, # lower the maximum seq length to 4096 instead of 8192 to fit in google colab GPUs.
        add_special_tokens={"pad_token": "[PAD]"}, # add pad token.
        chat_template=CHAT_TEMPLATE # add the new chat template including the system and input roles.
    )
)

## Vamos a Entrenar

Con nuestras configuraciones de dataset y modelo en su lugar, ahora estamos listos para iniciar el proceso de entrenamiento. Aquí es donde entra en juego la clase `AutoTrainer` de la biblioteca `autotransformers`, orquestando toda la operación de entrenamiento basada en las especificaciones que hemos proporcionado.

### Configurando el AutoTrainer

El `AutoTrainer` es una clase integral diseñada para agilizar el entrenamiento de modelos de aprendizaje automático, especialmente adaptada para modelos de lenguaje grandes. Acepta varios parámetros para controlar el proceso de entrenamiento:

- **Configuraciones del Modelo**: Una lista de objetos `ModelConfig`, cada uno definiendo las configuraciones y personalizaciones para un modelo. Para nuestro chatbot instructivo, incluimos la configuración para el modelo GEMMA adaptado con LoRA y cuantización.
- **Configuraciones del Dataset**: Similar a las configuraciones del modelo, estas se especifican usando objetos `DatasetConfig`. Pasamos la configuración para nuestro dataset `alpaca` preprocesado y estructurado, asegurando que se utilice efectivamente durante el entrenamiento.
- **Directorio de Métricas**: Especifica el directorio donde se almacenarán las métricas de entrenamiento, permitiendo el monitoreo y evaluación del rendimiento.
- **Modo de Búsqueda de Hiperparámetros**: Establecido en "fijo" en nuestro caso, indicando que no estamos explorando diferentes hiperparámetros sino entrenando con un conjunto predeterminado.
- **Limpieza**: Una bandera booleana para limpiar los datos de ejecuciones anteriores, asegurando un nuevo inicio para cada sesión de entrenamiento.
- **Limpiador de Métricas**: Especifica la utilidad para manejar datos temporales de métricas, manteniendo nuestro directorio de métricas ordenado y centrado en resultados significativos.
- **Usar Token de Autenticación**: Habilita el uso de un token de autenticación, necesario para acceder a ciertos modelos o datasets que pueden tener restricciones de acceso.

### Iniciando el Entrenamiento

Con el `AutoTrainer` configurado, procedemos a llamar a su método de ejecución. Este paso inicia el proceso de entrenamiento, aprovechando las configuraciones que hemos configurado meticulosamente. El proceso implica:

- Cargar y preparar automáticamente el dataset según nuestro `DatasetConfig`.
- Adaptar y afinar el modelo basado en el `ModelConfig`, incluyendo cualquier mejora de LoRA o cuantización especificada.
- Evaluar regularmente el rendimiento del modelo usando el conjunto de validación proporcionado, permitiéndonos monitorear su efectividad en tiempo real.
- Guardar puntos de control del modelo y métricas de entrenamiento, habilitando tanto la introspección del proceso de entrenamiento como la reanudación del entrenamiento desde el último estado guardado.

Al completarse, los resultados del entrenamiento, incluyendo métricas de rendimiento y puntos de control del modelo, están disponibles para análisis y despliegue. Este paso marca la culminación de la preparación de nuestro chatbot instructivo, dejándolo listo para pruebas y eventualmente, despliegue en escenarios del mundo real.


In [11]:
autotrainer = AutoTrainer(
    model_configs=[gemma_config], # NOTE: Aquí podríamos poner tantos modelos como quisiéramos, y se entrenarían en bucle.
    dataset_configs=[alpaca_config], # NOTE: Aquí también podríamos utilizar tantos datasets como quisiéramos.
    metrics_dir="./metrics_alpaca",
    hp_search_mode="fixed",
    use_auth_token=True
)

In [12]:
results = autotrainer()

Iterating over datasets...:   0%|          | 0/1 [00:00<?, ?it/s]




















Map (num_proc=4): 100%|██████████| 35320/35320 [00:05<00:00, 5994.24 examples/s]






Map (num_proc=4): 100%|██████████| 6233/6233 [00:02<00:00, 2563.99 examples/s]









Map (num_proc=4): 100%|██████████| 10389/10389 [00:03<00:00, 3419.05 examples/s]
`low_cpu_mem_usage` was None, now set to True since model is quantized.



Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.79s/it]
/home/alejandro/miniconda3/envs/test_autotransformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Iterating over datasets...:   0%|          | 0/1 [01:31<?, ?it/s]


KeyboardInterrupt: 